In [50]:
import numpy as np
import sympy as sym
import math
import matplotlib.pyplot as plt

In [56]:
def func(e,h,psi_order,lmax_order,d_order,wt,Display):
    w, psi, lmax, d = sym.symbols('w psi lmax d')
    
    #eqn = -psi + sym.atan((-50+w/2*sym.cos(psi)+0.5)/(-44-w/2*sym.sin(psi)))
    eqn = -psi + sym.atan((49.5 - w)/(44-w*sym.sin(psi)))
    psi_inst = []
    w0 = np.linspace(0,30,200)
    for i in w0:
        psi_inst.append(sym.nsolve(eqn.subs(w,i),psi,0.785))
    psi_inst = np.array(psi_inst)
    psi_inst = np.asarray(psi_inst, dtype=float)
    psi_inst = psi_inst*180/np.pi
    poly_coeff = np.polyfit(w0,psi_inst,psi_order)
    test_poly = np.polyval(poly_coeff, w0)
    #print(test_poly)
    w0.tofile('w0.csv', sep = ',')
    psi_inst.tofile('psi_inst.csv', sep = ',')
    psi_2_exact = sym.nsolve(eqn.subs({w:wt}), psi, 0.725)*180/np.pi
    if Display:
        display(eqn)
        print(poly_coeff)
        print([np.format_float_positional(i, precision=5, unique=False, fractional=False, trim='k') for i in poly_coeff])
        plt.plot(w0, psi_inst, label='numerical')
        plt.plot(w0, test_poly, label='fitted')
        plt.xlabel('w [mm]')
        plt.ylabel('psi/2 [deg]')
        plt.title('Psi/2 angle to maximize flexure length at each width')
        plt.grid(True)
        plt.legend()
        plt.show()
        print(f"psi/2 exact = {psi_2_exact:.2f}")
        print(f"psi/2 approx. = {np.polyval(poly_coeff, wt):.2f}")
        
    #print(w0)
    #print(psi_inst)
    
    #height
    height_inst = []
    height_eqn = -h + 21 + w*sym.sin(psi)+(3*lmax + 2*e)*sym.cos(psi)
    for i in w0:
        psi_inter = np.polyval(poly_coeff, i)*np.pi/180
        height_inst.append(sym.nsolve(height_eqn.subs({w:i, psi: psi_inter}),lmax,20))
    height_inst = np.array(height_inst)
    height_inst = np.asarray(height_inst, dtype=float)
    poly_coeff_h = np.polyfit(w0,height_inst,lmax_order)
    test_poly_h = np.polyval(poly_coeff_h, w0)
    height_inst.tofile('height_inst.csv', sep = ',')
    
    #print(w0)
    #print(height_inst)
    l_max_exact = sym.nsolve(height_eqn.subs({w: wt, psi: np.polyval(poly_coeff, wt) * np.pi / 180}), lmax, 20)
    if Display:
        display(height_eqn)
        print(poly_coeff_h)
        print([np.format_float_positional(i, precision=5, unique=False, fractional=False, trim='k') for i in poly_coeff_h])
        plt.plot(w0, height_inst, label = 'numerical')
        plt.plot(w0, test_poly_h, label='fitted')
        plt.xlabel('w [mm]')
        plt.ylabel('lmax [mm]')
        plt.title('Maximum flexure length at each width')
        plt.grid(True)
        plt.legend()
        plt.show()
        print(f"lmax exact = {l_max_exact:.2f}")
        print(f"lmax approx. = {np.polyval(poly_coeff_h, wt):.2f}")
    
    #d
    d_inst = []
    d_eqn = w/2*sym.cos(psi)+0.5
    for i in w0:
        psi_inter = np.polyval(poly_coeff, i)*np.pi/180
        d_inst.append(d_eqn.subs({w:i, psi: psi_inter}))
    d_inst = np.array(d_inst)
    d_inst = np.asarray(d_inst, dtype=float)
    poly_coeff_d = np.polyfit(w0,d_inst,d_order)
    test_poly_d = np.polyval(poly_coeff_d, w0)
    
    d_inst.tofile('d_inst.csv', sep = ',')
    
    #print(w0)
    #print(d_inst)
    d_2_exact = d_eqn.subs({w:wt, psi: np.polyval(poly_coeff, wt)*np.pi/180})
    
    if Display:
        display(d_eqn)
        print(poly_coeff_d)
        print([np.format_float_positional(i, precision=5, unique=False, fractional=False, trim='k') for i in poly_coeff_d])
        plt.plot(w0, d_inst, label = 'numerical')
        plt.plot(w0, test_poly_d, label='fitted')
        plt.xlabel('w [mm]')
        plt.ylabel('d/2 [mm]')
        plt.title('d/2 at each width')
        plt.grid(True)
        plt.legend()
        plt.show()
        print(f"d/2 exact.= {d_2_exact:.2f}")
        print(f"d/2 arrpox. = {np.polyval(poly_coeff_d, wt):.2f}")
    
    return [round(wt,2), round(psi_2_exact,2), round(l_max_exact,2), round(d_2_exact,2)]

#wt_list = np.linspace(5,10,2).tolist()
wt_list = [7.7]
parameters_list = []
header = 'w,psi/2,l_max,d/2'

for i in wt_list:
    parameters_list.append(func(e = 0.1, h = 65, psi_order = 5, lmax_order = 5, d_order = 5, wt = i, Display = False))

parameters_list = np.array(parameters_list)
np.savetxt('optimization_parameters.csv', parameters_list, delimiter=',',header = header, comments="", fmt='%.2f')
print(parameters_list)


[[7.7 47.48 18.84 3.10]]
